# Start - Datasets

In [ ]:
%%time

# Step1 - Get repo
! rm -rf CS4180-DL
! git clone --branch feature/prerak2 https://github.com/prerakmody/CS4180-DL

# Step2 - Get Dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_11-May-2012.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtest_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! mkdir /content/CS4180-DL/data/weights

! wget -P /content/CS4180-DL/data/weights https://pjreddie.com/media/files/yolov2-voc.weights

In [11]:
import os
import sys

%load_ext autoreload
%autoreload 2

## Step3 - Add path to main dir
DIR_MAIN = os.path.abspath('CS4180-DL/') #'../../../CS4180-DL/'
sys.path.append(DIR_MAIN)

## Step4 - Download pruned weights from GDrive
from src.utils import download_gdrive
FILE_ID =  '1dHnUQ8G3GObZSMh9eQ0zUR5wor0ttW3U'
DIR_WEIGHTS = os.path.join(DIR_MAIN, 'data/weights/pruned')
! mkdir {DIR_WEIGHTS}
DEST_NAME_ZIP   = os.path.join(DIR_WEIGHTS, 'weights-prune-me.zip')
DEST_NAME_FILES = DIR_WEIGHTS
download_gdrive(FILE_ID, DEST_NAME_ZIP)
! unzip {DEST_NAME_ZIP} -d {DEST_NAME_FILES}

## Step5 - Generate .txt files for training/validation 
from src.dataloader import setup_VOC
DIR_DATA = os.path.join(DIR_MAIN, 'data/dataset/')
setup_VOC(DIR_DATA)
DIR_DATA_VOC = os.path.join(DIR_DATA, 'VOCdevkit')
! cat {DIR_DATA_VOC}/2007_train.txt {DIR_DATA_VOC}/2007_val.txt {DIR_DATA_VOC}/2012_*.txt > {DIR_DATA_VOC}/voc_train.txt

## Step6 - Check which GPU
! nvidia-smi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 - year :  2012  || image_set :  train
 - year :  2012  || image_set :  val
 - year :  2007  || image_set :  train
 - year :  2007  || image_set :  val
 - year :  2007  || image_set :  test


# Start - Code

In [ ]:
import os
import sys

import torch

dir_main = os.path.abspath('CS4180-DL')
sys.path.append(dir_main)
print (' - In Path : ', sys.path[-1])

USE_GPU = torch.cuda.is_available()
print (' - USE_GPU : ', USE_GPU)

!nvidia-smi

# Start - Predict

In [ ]:
torch.cuda.empty_cache()
from src.predict import *
from src.nets import *

if (1):
    DIR_PROJ         = 'CS4180-DL'

if (1):
    MODEL            = ''
    MODEL_CFGFILE    = os.path.join(DIR_PROJ, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
    
    MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/yolov2-voc.weights') #['0.5' : 0.6366, '0.25' : 0.7, '0.1' : 0.7363]
    EVAL_PREFIX      = 'iter1_pretrained_'
    
    # MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/pruned/yolov2-voc-weight-prune-10.0.weights') #['0.5' : 0.6323, '0.25' : 0.7]
    # EVAL_PREFIX      = 'iter1_pretrained_10_'
    
    # MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/pruned/yolov2-voc-weight-prune-30.0.weights') #['0.5' : 0.6323, '0.25' : 0.69, ]
    # EVAL_PREFIX      = 'iter1_pretrained_30_'
    
    # MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/pruned/yolov2-voc-weight-prune-50.0.weights') #['0.25' :  0.6874]
    # EVAL_PREFIX      = 'iter1_pretrained_50_'
    
    # MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/pruned/yolov2-voc-weight-prune-70.0.weights') #['0.25' :  0.5160]
    # EVAL_PREFIX      = 'iter1_pretrained_70_'
    
    # MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/pruned/yolov2-voc-weight-prune-90.0.weights') #['0.25' :  0, '0.1' : 0, '0.005' : 0]
    # EVAL_PREFIX      = 'iter1_pretrained_90_'
    
    MODEL_LOSS       = RegionLoss()
    
    PASCAL_DIR       = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/')
    EVAL_IMAGELIST   = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/2007_test.txt')
    EVAL_OUTPUTDIR   = os.path.join(DIR_PROJ, 'eval_data')
    EVAL_OUTPUTDIR_PKL = os.path.join(DIR_PROJ, 'eval_results')
    
    
    # EVAL_PREFIX      = 'iter1_weighspruned_70_'
    
    print (' - 0. MODEL       : ', MODEL)
    print (' - 0. MODEL_WEIGHT: ', MODEL_WEIGHTFILE)
    print (' - 0. EVAL_PREFIX : ', EVAL_PREFIX)
   
if (1):
    BATCH_SIZE = 32
    CONF_THRESH = 0.05 # [0.5, 0.25, 0.1, 0.005]
    print (' - 0. BATCH_SIZE  : ', BATCH_SIZE)
    print (' - 0. CONF_THRESH : ', CONF_THRESH)
    
if (1):
    try:
        print (' - 0. Logger      : ', LOGGER)
    except:
        LOGGER = TensorBoardColab()
        print (' - 0. Logger      : ', LOGGER)    
    
    print ('')


valObj = PASCALVOCEval(MODEL, MODEL_CFGFILE, MODEL_WEIGHTFILE, MODEL_LOSS 
                       ,PASCAL_DIR, EVAL_IMAGELIST, EVAL_OUTPUTDIR, EVAL_PREFIX, EVAL_OUTPUTDIR_PKL
                       , LOGGER)
valObj.predict(BATCH_SIZE=BATCH_SIZE, CONF_THRESH=CONF_THRESH)
# valObj._do_python_eval()

# Start - Training

In [1]:
% reset
import os
import sys

import torch
from tensorboardcolab import TensorBoardColab

torch.cuda.empty_cache()



if (torch.cuda.is_available()):
    
    if (1):
        DIR_MAIN         = os.path.abspath('CS4180-DL')
        sys.path.append(DIR_MAIN)
        print (' - 0. DIR_MAIN     :  ', DIR_MAIN)
        from src.train import YOLOv2Train
        
    if (1):
        PASCAL_DIR   = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/')
        PASCAL_TRAIN = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/voc_train.txt')
        PASCAL_VALID = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/2007_test.txt')
        TRAIN_LOGDIR = os.path.join(DIR_MAIN, 'train_data')
        VAL_LOGDIR   = os.path.join(DIR_MAIN, 'eval_data')
        VAL_OUTPUTDIR_PKL = os.path.join(DIR_MAIN, 'eval_results')
        MODEL_CFG    = os.path.join(DIR_MAIN, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
        
    if (1):
        MODEL_WEIGHT = os.path.join(DIR_MAIN, 'data/weights/pruned/yolov2-voc-weight-prune-30.0.weights')
        VAL_PREFIX   = 'pretrained-30_'
        
        # MODEL_WEIGHT = os.path.join(DIR_MAIN, 'data/weights/yolov2-voc.weights')
        # VAL_PREFIX   = 'pretrained_'
        
        print (' - 0. MODEL_WEIGHT :  ', MODEL_WEIGHT)
        print (' - 0. VAL_PREFIX   : ', VAL_PREFIX)
        
    if (1):
        BATCH_SIZE   = 32;
        print (' - 0. BATCH_SIZE   : ', BATCH_SIZE)

    if (1):
        DEBUG_EPOCHS = 50 #[-1, 50]
        print (' - 0. DEBUG_EPOCHS : ', DEBUG_EPOCHS)
        try:
            print (' - 0. Logger       : ', LOGGER)
        except:
            LOGGER = TensorBoardColab()
            print (' - 0. Logger       : ', LOGGER)
        print ('')


    if (1):
        trainObj = YOLOv2Train()
        trainObj.train(PASCAL_DIR, PASCAL_TRAIN, PASCAL_VALID, TRAIN_LOGDIR, VAL_LOGDIR, VAL_OUTPUTDIR_PKL, VAL_PREFIX
                       , MODEL_CFG, MODEL_WEIGHT
                       , BATCH_SIZE
                       , LOGGER, DEBUG_EPOCHS=DEBUG_EPOCHS, verbose=0)
else:
    print (' - GPU Issues!!')

UsageError: Line magic function `%` not found.
